# Fake News Detection Engine

Step1: Import all the neccesary libraries

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score

from sklearn.model_selection import cross_val_score

C:\Users\91847\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Step2: Reading the training dataset 

In [5]:
df = pd.read_csv("data used/train.csv")

In [6]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


Step3: We will change the terminology a little bit, in the label column, we will change 0 with Real and 1 with Fake as that is what they originally represent  

In [7]:
conversion_dict = { 0:'Real',
                    1:'Fake'}

df['label'] = df['label'].replace(conversion_dict)

df.label.value_counts() #to check the balance of dataset

Fake    10413
Real    10387
Name: label, dtype: int64

In [8]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Fake
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,Real
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Fake
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Fake
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Fake
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Real
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,Real
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Real
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",Fake


Step4: We will divide our data in training and testing frames and vectorize for the same 

In [10]:
x_train,x_test,y_train,y_test = train_test_split(df['text'],df['label'],
                                                test_size = 0.25,
                                                random_state = 7,
                                                shuffle = True)

tfidf_vectorizer = TfidfVectorizer(stop_words='english', 
                                  max_df=0.75)

In [11]:
vec_train = tfidf_vectorizer.fit_transform(x_train.values.astype('U'))

vec_test = tfidf_vectorizer.transform(x_test.values.astype('U'))

In [13]:
pac = PassiveAggressiveClassifier(max_iter=50)  # Our one of the best tools, the PAC in the house ;
pac.fit(vec_train,y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

Step5: We will predict accuracy 

In [14]:
y_pred = pac.predict(vec_test)

score = accuracy_score(y_test,y_pred)

print(f'PAC Accuracy: {round(score*100,2)}%')

PAC Accuracy: 96.15%


In [17]:
confusion_matrix(y_test, y_pred, labels = ['Real','Fake'])


array([[2484,  102],
       [  98, 2516]], dtype=int64)

K Fold Cross Validation Accuracy test (The Gold Standard of ML models)

In [19]:
X = tfidf_vectorizer.transform(df['text'].values.astype('U'))

In [22]:
scores = cross_val_score(pac, X, df['label'].values, cv=5)
print(f'K Fold Accuracy: {round(scores.mean()*100,2)}%')

K Fold Accuracy: 96.25%


Step6: Remember we kept aside a dataset so that when we use it, it should be completely new to our model. It is time for it to be used in our stress test 

In [28]:
df_true = pd.read_csv("True.csv")
df_false = pd.read_csv("Fake.csv")

df_true['label'] = 'Real'
df_true_rep=[df_true['text'][i].replace('WASHINGTON (Reuters) - ','').replace('LONDON (Reuters) - ','').replace('(Reuters) - ','') for i in range(len(df_true['text']))]
df_true['text']=df_true_rep
df_false['label']='Fake'
df_final=pd.concat([df_true,df_false])
df_final=df_final.drop(['subject','date'], axis=1)
df_false



,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",Fake
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",Fake
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",Fake
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",Fake


We will make a new function to test the data and custom user defined input 

In [29]:
def findlabel(userinput):
    vec_userinput = tfidf_vectorizer.transform([userinput])
    y_pred1 = pac.predict(vec_userinput)
    return y_pred1[0]

In [36]:
findlabel(""" Jaguar Land Rover and the University of Cambridge have collectively developed a new contactless touchscreen technology that is said to make the driving safer by eliminating the need to look at the screen and helping the driver concentrate better on the road. This technology is also claimed to help reduce the spread of bacteria and viruses in a post Covid-19 world.

The new contactless touchscreen technology has been named Predictive Touch and it employs artificial intelligence and sensors which predict the users intended target over the screen. The company claims that features such as satellite navigation, temperature controls, entertainment setting and more can be accessed without touching a button.

(Also Read: Jaguar Land Rover crosses 15-lakh production mark for Ingenium engine range)

As per official claims, the lab-tests and on-road trials have showed that the Predictive Touch technology can help reduce a users touchscreen interaction effort and time by up to 50%, alongside limiting the spread of bacteria and viruses.

This system is part of the Jaguar Land Rovers Destination Zero vision and has been developed with engineers at the University of Cambridge.

Lee Skrypchuk, Human Machine Interface Technical Specialist, at Jaguar Land Rover, said: As countries around the world exit lockdown, we notice how many everyday consumer transactions are conducted using touchscreens: railway or cinema tickets, ATMs, airport check-ins and supermarket self-service checkouts, as well as many industrial and manufacturing applications. Predictive Touch technology eliminates the need to touch an interactive display and could therefore reduce the risk of spreading bacteria or viruses on surfaces.

The new Predictive Touch technology will join the list of other innovative features by Jaguar Land Rover such as Driver Condition Monitor, engine noise cancellation and cabin air ionisation with PM 2.5 filtration to capture ultrafine particles and allergens, that have been developed keeping the passenger safety and well-being in mind.

(Also Read: Land Rover adds hybrid diesel model to plug-in electric Range Rover)

Skrypchuk added,The technology also offers us the chance to make vehicles safer by reducing the cognitive load on drivers and increasing the amount of time they can spend focused on the road ahead. This is a key part of our Destination Zero journey.""")

'Real'